#  주택청약 FAQ 시스템 챗봇 구현 

- 문서 전처리 + RAG + Gradio ChatInterface

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-nano',
    temperature=0.1,
    top_p=0.9, 
)

/Users/kwonsoonhyun/Sesac/004_llm_agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.4.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/kwonsoonhyun/Sesac/004_llm_agent/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_

# **문서 전처리 파이프라인**


### 1) 문서 로드

- 국토교통부 주택청약 FAQ에서 일부 내용(청약자격, 청약통장)을 발췌하여 재가공
- Q1 ~ Q50까지 모두 50개의 문답이 포함된 텍스트 파일

In [5]:
# 파일 경로 설정
faq_text_file = "data/housing_faq.txt"

# 파일 읽기 - 파이썬 내장 함수 사용
with open(faq_text_file, 'r') as f:
    faq_text = f.read()

# 파일 내용 확인
print(faq_text[:500])

Q1 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
A 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 
참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

Q2 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
A 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다.
* 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원
다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 


##### ***[실습] TextLoader를 사용하여, 텍스트 문서를 로드합니다.*** 

<details>
<summary>💡 정답 보기</summary>

```python
from langchain_community.document_loaders import TextLoader

# TextLoader 클래스를 사용하여 FAQ 텍스트 파일을 로드
loader = TextLoader(faq_text_file)  
docs = loader.load()
```
</details>


In [10]:
from langchain_community.document_loaders import TextLoader

# TextLoader 클래스를 사용하여 FAQ 텍스트 파일을 로드
loader = TextLoader(faq_text_file)  
docs = loader.load()

In [11]:
# 문서 확인
print(docs[0].page_content[:500])

Q1 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
A 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 
참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

Q2 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
A 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다.
* 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원
다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 


In [12]:
# 문서 메타데이터 확인
docs[0].metadata

{'source': 'data/housing_faq.txt'}

### 2) 문서 전처리

`(1) 각 질문과 답변을 쌍으로 추출하여 정리 (정규표현식 활용)`

In [13]:
import re

def extract_qa_pairs(text):
    qa_pairs = []
    
    # 텍스트를 라인별로 분리하고 각 라인의 앞뒤 공백 제거
    lines = [line.strip() for line in text.split('\n')]
    current_question = None
    current_answer = []
    current_number = None
    in_answer = False
    
    for i, line in enumerate(lines):
        if not line:  # 빈 라인 처리
            if in_answer and current_answer and i + 1 < len(lines) and lines[i + 1].startswith('Q'):
                # 다음 질문이 시작되기 전 빈 줄이면 현재 QA 쌍 저장
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
                in_answer = False
                current_answer = []
            continue
            
        # 새로운 질문 확인 (Q 다음에 숫자가 오는 패턴)
        q_match = re.match(r'Q(\d+)\s+(.*)', line)
        if q_match:
            # 이전 QA 쌍이 있으면 저장
            if current_question is not None and current_answer:
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
            
            # 새로운 질문 시작
            current_number = int(q_match.group(1))
            current_question = q_match.group(2).strip().rstrip('?') + '?'  # 질문 마크 정규화
            current_answer = []
            in_answer = False
            
        # 답변 시작 확인
        elif line.startswith('A ') or (current_question and not current_answer and line):
            in_answer = True
            current_answer.append(line.lstrip('A '))
            
        # 기존 답변에 내용 추가
        elif current_question is not None and (in_answer or not line.startswith('Q')):
            if in_answer or (current_answer and not line.startswith('Q')):
                current_answer.append(line)
    
    # 마지막 QA 쌍 처리
    if current_question is not None and current_answer:
        qa_pairs.append({
            'number': current_number,
            'question': current_question,
            'answer': ' '.join(current_answer).strip()
        })
    
    # 번호 순서대로 정렬
    qa_pairs.sort(key=lambda x: x['number'])
    
    return qa_pairs

In [14]:
# QA 쌍 추출
qa_pairs = extract_qa_pairs(docs[0].page_content) 

print(f"추출된 QA 쌍 개수: {len(qa_pairs)}")
print(f"추출된 첫번째 QA: \n{qa_pairs[0]}")

추출된 QA 쌍 개수: 50
추출된 첫번째 QA: 
{'number': 1, 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?', 'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.'}


`(2) LLM으로 추가 정보를 추출`
- 텍스트에서 키워드와 핵심 개념을 추출하는 체인
- 메타데이터 or 본문(page_content)에 추가하여 검색에 활용    

##### ***[실습] 다음 모델 스키마를 사용하여, 키워드와 요약을 추출하는 체인을 완성합니다.*** 

In [ ]:
# QA 쌍 확인 (첫번째) 질문 답변이 있는데, 핵심키워드 하나 뽑아볼거다 간단한 텍스트 요약을 뽑아올거다.
print((qa_pairs[0]['question']+"\n\n"+qa_pairs[0]['answer']))

경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?

해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 출력 형식 정의 파이덴틱 베이스모델 기반으로 자료 출력구조 
class KeywordOutput(BaseModel):
    keyword: str = Field(description="텍스트에서 추출한 가장 중요한 키워드(법률용어, 주제 등))")
    summary: str = Field(description="텍스트의 간단한 요약")

# 프롬프트 템플릿 정의
template = None

# LCEL 체인 구성 (Sturctured Output 사용)
prompt = None
llm_with_structure = None 
keyowrd_extractor = None | None

# 텍스트 추출 테스트     
result = keyowrd_extractor.invoke(qa_pairs[0]['question']+qa_pairs[0]['answer'])
print("키워드:", result.keywords)
print("요약:", result.summary)

<details>
<summary>💡 정답 보기</summary>

```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 출력 형식 정의
class KeywordOutput(BaseModel):
    keyword: str = Field(description="텍스트에서 추출한 가장 중요한 키워드(법률용어, 주제 등))")
    summary: str = Field(description="텍스트의 간단한 요약(한 문장)")

# 프롬프트 템플릿 정의
template = """주어진 텍스트에서 중요한 키워드를 추출하고, 텍스트의 간단한 요약을 작성합니다.
텍스트의 맥락을 고려하여 핵심 용어나 전문 용어, 주요 아이디어나 원리 등을 추출합니다. 
추출하는 키워드는 하나로 제한하며, 텍스트의 주제를 잘 나타내는 단어를 선택합니다.

텍스트:
{input_text}

JSON 형식으로 다음 정보를 반환하시오:
- keyword: 핵심 키워드 (1개)
- summary: 텍스트의 간단한 요약 (1-2문장)
"""

# LCEL 체인 구성
prompt = ChatPromptTemplate.from_template(template)
llm_with_structure = llm.with_structured_output(KeywordOutput)  #llm의 출력
keyowrd_extractor = prompt | llm_with_structure

# 텍스트 추출 테스트     
result = keyowrd_extractor.invoke(qa_pairs[0]['question']+"\n\n"+qa_pairs[0]['answer'])
print("키워드:", result.keyword)
print("요약:", result.summary)
```
</details>

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 출력 형식 정의
class KeywordOutput(BaseModel):
    keyword: str = Field(description="텍스트에서 추출한 가장 중요한 키워드(법률용어, 주제 등))")
    summary: str = Field(description="텍스트의 간단한 요약(한 문장)")

# 프롬프트 템플릿 정의
template = """주어진 텍스트에서 중요한 키워드를 추출하고, 텍스트의 간단한 요약을 작성합니다.
텍스트의 맥락을 고려하여 핵심 용어나 전문 용어, 주요 아이디어나 원리 등을 추출합니다. 
추출하는 키워드는 하나로 제한하며, 텍스트의 주제를 잘 나타내는 단어를 선택합니다.

텍스트:
{input_text}

JSON 형식으로 다음 정보를 반환하시오:
- keyword: 핵심 키워드 (1개)
- summary: 텍스트의 간단한 요약 (1-2문장)
"""

# LCEL 체인 구성
prompt = ChatPromptTemplate.from_template(template)
llm_with_structure = llm.with_structured_output(KeywordOutput)  #llm의 출력
keyowrd_extractor = prompt | llm_with_structure

# 텍스트 추출 테스트     
result = keyowrd_extractor.invoke(qa_pairs[0]['question']+"\n\n"+qa_pairs[0]['answer'])
print("키워드:", result.keyword)
print("요약:", result.summary)

키워드: 주택건설지역
요약: 이 텍스트는 경기도 과천시에서 공급되는 주택의 건설지역 범위와 관련 행정구역에 대해 설명하고 있습니다.


`(3) QA 쌍을 문자열 포맷팅하고 문서 객체로 변환`

In [ ]:
from langchain_core.documents import Document #방금 포매팅된 문자열 = 질문번호, 질문답변 페이지 컨텐트를 추가하고 요약문 추가한것들을 메타데이터로 넣는다. 
#그걸 리스트로 담아서 50개의 반복문이 있다.

def format_qa_pairs(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    """
    processed_docs = []
    for pair in qa_pairs:

        # QA 쌍을 포맷팅
        formatted_output = (
            f"[{pair['number']}]\n"
            f"질문: {pair['question']}\n"
            f"답변: {pair['answer']}\n"
        )

        # 키워드와 요약 추출
        result = keyowrd_extractor.invoke(pair['question']+"\n\n"+pair['answer'])

        # 문서 객체 생성
        doc = Document(
            page_content=formatted_output,
            metadata={
                'question_id': int(pair['number']),
                'question': pair['question'],
                'answer': pair['answer'],
                'keyword': result.keyword,
                'summary': result.summary
            }
        )
        processed_docs.append(doc)

    return processed_docs


# QA 쌍 포맷팅
formatted_docs = format_qa_pairs(qa_pairs)
print(f"포맷팅된 문서 개수: {len(formatted_docs)}")

# 문서 확인
print(formatted_docs[0].page_content)
print("-" * 200)
# 문서 메타데이터 확인
pprint(formatted_docs[0].metadata)

In [ ]:
# 문서 저장
output_file = "data/housing_faq_formatted.json"

#랭체인 다큐먼트 되있는거를 제이슨으로 바꾸는 형식이로 문서들으 저장한거다.
with open(output_file, 'w', encoding='utf-8-sig') as f:
    json.dump([doc.model_dump() for doc in formatted_docs], f, indent=2, ensure_ascii=False)  # 한글이 유니코드로 변환되지 않도록 설정
print(f"포맷팅된 문서를 {output_file}에 저장했습니다.")

##### ***[실습] 문서 객체를 포맷팅하여 구성합니다.*** 

- 요약문을 시맨틱 검색에 활용합니다. 다음 구조로 문서 객체를 생성합니다. 
    - page_content: 요약
    - metadata: 기타 정보

In [ ]:
# 여기에 코드를 작성하세요. 이건 요약문을 다르게 구성해놓은 실습이다.


def format_qa_pairs_with_summary(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    """


<details>
<summary>💡 정답 보기</summary>

```python
from langchain_core.documents import Document

def format_qa_pairs_with_summary(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    """
    processed_docs = []
    for pair in qa_pairs:

        # 키워드와 요약 추출
        result = keyowrd_extractor.invoke(pair['question']+"\n\n"+pair['answer'])

        # 문서 객체 생성
        doc = Document(
            page_content=result.summary,
            metadata={
                'question_id': int(pair['number']),
                'question': pair['question'],
                'answer': pair['answer'],
                'keyword': result.keyword,
            }
        )
        processed_docs.append(doc)

    return processed_docs
```
</details>

In [ ]:
# QA 쌍 포맷팅
summary_formatted_docs = format_qa_pairs_with_summary(qa_pairs) 
print(f"포맷팅된 문서 개수: {len(summary_formatted_docs)}")

# 문서 확인
print(summary_formatted_docs[0].page_content)
print("-" * 200)
# 문서 메타데이터 확인
pprint(summary_formatted_docs[0].metadata)

In [ ]:
# 문서 저장
output_file = "data/housing_faq_formatted_with_summary.json"

with open(output_file, 'w', encoding='utf-8-sig') as f:
    json.dump([doc.model_dump() for doc in summary_formatted_docs], f, indent=2, ensure_ascii=False)  # 한글이 유니코드로 변환되지 않도록 설정
print(f"포맷팅된 문서를 {output_file}에 저장했습니다.")

# 벡터 저장 

In [ ]:
# 문서 로드 제이슨형식으로된걸 읽어와서 이건 리스트니까 하나의 아이템들을 페이지컨텐트속성으로 메타데이터 속성을 지정 키밸류로 되있는거를 갖는상태로 변환해준다

from langchain_core.documents import Document

output_file = "data/housing_faq_formatted.json"

#제이슨으로 저장된거 불러온거다 읽어온거다.
with open(output_file, 'r', encoding='utf-8-sig') as f:
    formatted_docs = [Document(**doc) for doc in json.load(f)]

print(len(formatted_docs))

In [17]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 문서 벡터 저장 크로마에 컬렉션이름 새로 지정해서 저장을 할거다. 50개의 문서가 저장될거다.
vector_store = Chroma.from_documents(  
    documents=formatted_docs,
    embedding=embeddings,
    collection_name="housing_faq_db",
    persist_directory="./chroma_db",
)

NameError: name 'formatted_docs' is not defined

In [ ]:
vector_store._collection.count()

##### ***[실습] 요약 문서(summary_formatted_docs)를 벡터 스토어에 저장합니다.*** 

- OpenAI (text-embedding-3-small) 임베딩 모델 사용
- Chromda DB 사용

In [ ]:
# 문서 로드
from langchain_core.documents import Document

output_file = "data/housing_faq_formatted_with_summary.json"

with open(output_file, 'r', encoding='utf-8-sig') as f:
    summary_formatted_docs = [Document(**doc) for doc in json.load(f)]

# 문서 확인 동일한 문서 읽어올수있고
print(summary_formatted_docs[0].page_content)

# 문서 메타데이터 확인
pprint(summary_formatted_docs[0].metadata)

In [ ]:
# 여기에 코드를 작성하세요.


<details>
<summary>💡 정답 보기</summary>

```python
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 문서 벡터 저장
vector_store_with_summary = Chroma.from_documents(  
    documents=summary_formatted_docs,
    embedding=embeddings,
    collection_name="housing_faq_summary_db",
    persist_directory="./chroma_db",
)

vector_store_with_summary._collection.count()
```
</details>

# 문서 검색

- 벡터 저장소 로드

In [ ]:
#3개의 문서읽어오게해서

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소 로드
vector_store = Chroma(
    collection_name="housing_faq_db",
    persist_directory="./chroma_db", 
    embedding_function=embeddings,
)

In [ ]:
print(f"벡터 저장소에 저장된 문서 개수: {vector_store._collection.count()}")

##### ***[실습] 앞에서 저장한 벡터 스토어를 로드합니다.*** 

- OpenAI (text-embedding-3-small) 임베딩 모델 사용
- Chromda DB 사용

In [ ]:
# 여기에 코드를 작성하세요.


# 벡터 저장소 로드
vector_store_with_summary = None

<details>
<summary>💡 정답 보기</summary>

```python
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소 로드
vector_store_with_summary = Chroma(
    collection_name="housing_faq_summary_db",
    persist_directory="./chroma_db", 
    embedding_function=embeddings,
)

print(f"벡터 저장소에 저장된 문서 개수: {vector_store_with_summary._collection.count()}")
```
</details>

- 벡터 저장소를 리트리터 검색도구로 변환

In [ ]:
# 검색기 생성 - 유사도 기반 상위 3개 문서 검색 바로아래 그냥 리트리버만들어놓음
retriever = vector_store.as_retriever(
    search_kwargs={"k": 3},
)
#llm이 이 내용을 가지고, 과천시랑 수원시랑 1대1대응해서 대답하는데, 이걸 이해못하면 답변생성에 실패한다.


# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

##### ***[실습] MMR 검색기를 정의합니다.*** 

- 요약 문서 벡터 스토어를 사용
- 10개의 문서를 가져와서, 다양성 기반으로 3개를 선택 (다양성은 중간 수준 적용)

In [ ]:
# 여기에 코드를 작성하세요.

mmr_retriever = None

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = mmr_retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print(result.metadata['question'])
    print(result.metadata['answer'])
    print("=" * 50)

<details>
<summary>💡 정답 보기</summary>

```python
# 검색기 생성 - 유사도 기반 상위 3개 문서 검색
mmr_retriever = vector_store_with_summary.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 10, "lambda_mult": 0.5},
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = mmr_retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print(result.metadata['question'])
    print(result.metadata['answer'])
    print("=" * 50)
```
</details>

### **[심화] 메타데이터 기반 필터링**

- Chroma 문서: https://docs.trychroma.com/docs/querying-collections/metadata-filtering

In [18]:
# 단일 필드 정확히 일치 #실무에서 가장 많이 사용됨. 중요. 메타데이터 길이에따라 달라진다.
#컨텍스트 벡터스토어를 새로운 데이터를 넣어서 업데이트하면, 똑같은 쿼리를 넣어도 검색결과가 달라진다. 이 문제를 해결할 방법은 월별로 따로 만든다던지, 카테고리별로 따로 만들어놓는다던지
# 벡터저장소가 여러군데 분리가 돼을건데 쿼리가 격리돼서 저장하는 기법 중요. 크로마나 일라서치로 저장소 따로도 있고, 날짜, 카테고리화. 잘정리해서 라우팅을 잘 하도록해야한다.

#메타데이터는 격리는 아니고, 그안에서 메타데이터안에서 연도라던지 카테고리마다 정리해두면 카테고리에 해당하는 데이터만 검색해오면 
#필터를 지원을 한다.
retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": "해당 주택건설지역"}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

NameError: name 'vector_store' is not defined

In [ ]:
# $eq 연산자 사용 - 정확히 일치, 두번째방법은 이퀄연산자 사용한다. 크로마 벡터디비에서는 기본적으로 검색해와서 필터를 한다. 너무커진다싶으면 디비를 여러개로 분할한다.
#메타데이터 필터링은 전체검색해서 필터하는데, 애초에 나뉜거만 검색하는 것도 개발중이다.

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$eq": "해당 주택건설지역"}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $ne (Not Equal) 연산자 사용 - 정확히 일치하지 않는 문서 검색, 이방법은 아닌 문서만 검색하는거다.

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$ne": "해당 주택건설지역"}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $in 연산자로 여러 값 중 일치하는 문서 검색 

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"keyword": {"$in": ["해당 주택건설지역", "청약예금"]}}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# 숫자 범위 검색 ($gt, $gte, $lt, $lte) - question_id가 10 이상인 문서 검색 
#숫자로된거는 범위지정을 할 수가 있다.

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"question_id": {"$gte": 10}}},
)

query = "무주택자 기준은 무엇인가요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $and로 여러 조건 조합 - keyword가 "해당 주택건설지역"이고 question_id가 10 미만인 문서 검색
#질문번호가 10보다 작은거

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"$and": [
        {"keyword": "해당 주택건설지역"}, 
        {"question_id": {"$lt": 10}}
    ]}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# $or로 여러 조건 중 하나 일치하는 문서 검색 - keyword가 "해당 주택건설지역"이거나 question_id가 10 이상인 문서 검색
#or은 둘중에 하나라도 포함돼잇는거

retriever = vector_store.as_retriever(
    search_kwargs={"filter": {"$or": [
        {"keyword": "해당 주택건설지역"}, 
        {"question_id": {"$gte": 10}}
    ]}},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

In [ ]:
# 정규식 패턴 매칭 - page_content 본문에 "주택건설지역"이 포함된 문서 검색

retriever = vector_store.as_retriever(
    search_kwargs={'where_document': {'$contains': '주택건설지역'}},
) #문자열이 포함되는 지역을 체크해주는 본문검색기능도 부분적으로는 적용한다.

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

- 메타데이터 필터링 체인 (`lark` 패키지 설치 필요)

In [ ]:
results[0].metadata

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import chain
from langchain_core.documents import Document
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any, Union
import re
import json

#질문하자마자 질문하는거를 필터로 나눠주는 역할을 만들고싶엇다.

class QueryFilter(BaseModel):
    """쿼리 필터 정보를 담는 모델"""
    semantic_query: str = Field(description="의미 검색에 사용할 텍스트")
    keyword: Optional[str] = Field(default=None, description="키워드 필터")
    question_id_min: Optional[int] = Field(default=None, description="question_id 최소값")
    question_id_max: Optional[int] = Field(default=None, description="question_id 최대값")
    limit: Optional[int] = Field(default=None, description="결과 개수 제한") #k 값 정하려고 만들어놓음

class CustomQueryRetriever:
    """
    SelfQueryRetriever를 대체하는 커스텀 리트리버
    LLM으로 구조화된 정보를 추출하고 직접 Chroma 필터를 생성한다
    """
    
    def __init__(self, vectorstore, llm=None):
        self.vectorstore = vectorstore
        self.llm = llm or ChatOpenAI(model='gpt-4.1-mini', temperature=0)
        self.query_parser = self._create_query_parser()
    
    def _create_query_parser(self):
        """구조화된 쿼리 파서 생성"""#최종줄력은 쿼리 파서필터
        
        system_prompt = """사용자의 자연어 쿼리를 분석하여 검색에 필요한 정보를 구조화한다.

다음 규칙을 따른다:
1. semantic_query: 의미 검색에 사용할 핵심 키워드나 내용
2. keyword: 정확히 일치해야 하는 키워드 (있는 경우만)
3. question_id_min/max: 질문 ID 범위 (예: "10번 이하" → max=10, "5번 이상" → min=5)
4. limit: 검색 결과 개수 제한 (기본값은 5개)

예시:
- "'해당 주택건설지역' 관련 문서를 10번 이하인 문서중에서 검색해주세요"
  → semantic_query: "주택건설지역", keyword: "해당 주택건설지역", question_id_max: 10

- "'주택공급' 키워드가 포함된 문서를 5개 검색해주세요"
  → semantic_query: "주택공급", keyword: "주택공급", limit: 5

정보가 없는 필드는 null로 설정한다."""

        prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("human", "다음 쿼리를 분석해주세요: {query}")
        ]) #시스템프롬프트에 자세히 다 씀
        
        return prompt | self.llm.with_structured_output(QueryFilter)
    
    def _build_chroma_filter(self, filter_info: QueryFilter) -> Dict[str, Any]:
        """QueryFilter 정보를 Chroma DB 필터로 변환"""
        conditions = []
        
        # 키워드 필터
        if filter_info.keyword:
            conditions.append({"keyword": {"$eq": filter_info.keyword}})
        
        # question_id 범위 필터
        if filter_info.question_id_min is not None and filter_info.question_id_max is not None:
            conditions.append({
                "$and": [
                    {"question_id": {"$gte": filter_info.question_id_min}},
                    {"question_id": {"$lte": filter_info.question_id_max}}
                ]
            }) #최솟값만 있으면 이상 최댓갓만잇으면 이하로
        elif filter_info.question_id_min is not None:
            conditions.append({"question_id": {"$gte": filter_info.question_id_min}})
        elif filter_info.question_id_max is not None:
            conditions.append({"question_id": {"$lte": filter_info.question_id_max}})
        
        # 조건들을 AND로 결합
        if len(conditions) == 0:
            return {}
        elif len(conditions) == 1:
            return conditions[0]
        else:
            return {"$and": conditions}
    
    def invoke(self, query: str) -> List[Document]:
        """쿼리 실행""" #커스텀 리트리버로 만드렀고 여기 리트리버 부분 인보크로 실행하면
        try:
            # 쿼리 분석 쿼리가들어오면 필터정보를 추출 추출된거로 그로마필터 생성
            filter_info = self.query_parser.invoke({"query": query})
            print(f"🔍 분석된 쿼리: {filter_info}")
            
            # Chroma 필터 생성
            chroma_filter = self._build_chroma_filter(filter_info)
            print(f"🔧 생성된 필터: {chroma_filter}")
            
            # 검색 실행
            search_kwargs = {}
            
            if chroma_filter:
                search_kwargs["filter"] = chroma_filter
            
            k = filter_info.limit or 5 #리밋을 지정
            search_kwargs["k"] = k
            
            # 의미 검색 쿼리 결정
            search_query = filter_info.semantic_query or query
            #벡터스토어에 유사도검색사용
            results = self.vectorstore.similarity_search(
                query=search_query,
                **search_kwargs
            ) #llm이 생성하도록 만듬
            
            print(f"✅ 검색 완료: {len(results)}개 문서 발견")
            return results
            
        except Exception as e:
            print(f"❌ 검색 오류: {e}")
            # 실패 시 기본 검색
            return self.vectorstore.similarity_search(query, k=5)



@chain #쿼리가 들어오면 리트리버체인이고 문서가 검색된결과가 리턴이되고 
def smart_custom_search(query: str) -> List[Document]:
    """
    커스텀 리트리버를 사용한 스마트 검색
    """
    print(f"🚀 검색 시작: {query}")
    
    # 1차 시도: LLM 기반 커스텀 리트리버
    try:
        print("🔄 LLM 기반 분석 시도...") #앞에 커스텀리트리버만든걸통해서 검색하면 그결과가 리턴해주게만듬
        llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0)
        retriever = CustomQueryRetriever(vector_store, llm)
        results = retriever.invoke(query)
        if results:
            return results #실행중에 에러발생하면
    except Exception as e:
        print(f"⚠️ LLM 기반 검색 실패: {e}")
    
    # 2차 시도: 기본 검색 #일반검색으로
    try:
        print("🔄 기본 검색 실행...")
        results = vector_store.similarity_search(query, k=5)
        print(f"✅ 기본 검색 성공: {len(results)}개 문서")
        return results
    except Exception as e:
        print(f"❌ 모든 검색 실패: {e}")
        return []



# 사용 예시 및 테스트 #테스트질문
test_queries = [
    "'해당 주택건설지역' 키워드가 포함된 문서를 10번 이하인 문서중에서 검색해주세요",
    "'청약예금' 키워드가 포함된 문서들",
    "'질문 ID'가 5에서 15 사이인 문서",
]

for query in test_queries:
    print(f"\n📝 테스트 쿼리: {query}")
    print("-" * 100)
    
    try:
        # 커스텀 검색 실행
        results = smart_custom_search.invoke(query)
        
        # 결과 출력
        for i, result in enumerate(results[:2], 1):  # 처음 2개만
            print(f"{i}. {result.page_content[:60]}...")
            print(f"   📋 메타데이터: {result.metadata}")
            
    except Exception as e:
        print(f"❌ 테스트 실패: {e}")
    
    print("=" * 100)


📝 테스트 쿼리: '해당 주택건설지역' 키워드가 포함된 문서를 10번 이하인 문서중에서 검색해주세요
----------------------------------------------------------------------------------------------------
🚀 검색 시작: '해당 주택건설지역' 키워드가 포함된 문서를 10번 이하인 문서중에서 검색해주세요
🔄 LLM 기반 분석 시도...
⚠️ LLM 기반 검색 실패: name 'vector_store' is not defined
🔄 기본 검색 실행...
❌ 모든 검색 실패: name 'vector_store' is not defined

📝 테스트 쿼리: '청약예금' 키워드가 포함된 문서들
----------------------------------------------------------------------------------------------------
🚀 검색 시작: '청약예금' 키워드가 포함된 문서들
🔄 LLM 기반 분석 시도...
⚠️ LLM 기반 검색 실패: name 'vector_store' is not defined
🔄 기본 검색 실행...
❌ 모든 검색 실패: name 'vector_store' is not defined

📝 테스트 쿼리: '질문 ID'가 5에서 15 사이인 문서
----------------------------------------------------------------------------------------------------
🚀 검색 시작: '질문 ID'가 5에서 15 사이인 문서
🔄 LLM 기반 분석 시도...
⚠️ LLM 기반 검색 실패: name 'vector_store' is not defined
🔄 기본 검색 실행...
❌ 모든 검색 실패: name 'vector_store' is not defined


# RAG Chain

### 1) 참조 문서 없이 직접 답변을 생성

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Prompt #초안은 한국어래도, 프롬프트는 영어로 한다. 그리고 
template = '''Answer the question based only on the following context.

[Context]
{context}

[Question]
{question}

[Answer (in 한국어)]
'''

prompt = ChatPromptTemplate.from_template(template)


# 문서 포맷팅
def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])


# 검색기 생성 - 유사도 기반 상위 5개 문서 검색
retriever = vector_store.as_retriever(
    search_kwargs={"k": 5},
) #벡터스토어리트리버


# Chain 구성
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
query = "수원시의 주택건설지역은 어디에 해당하나요?"
rag_chain.invoke(query)

### 2) 참조 문서를 답변과 함께 반환

`(1) 문서와 포맷팅된 컨텍스트를 함께 반환하는 함수`


In [ ]:
# 다음 코드를 완성하세요. 두번째는 rag면 리트리버가검색해온 결과 코드를 , 웹이면 검색결과 다 보여주는 걸 사용자에게 보여주는 코드

from typing import Dict

def get_context_and_docs(question: str) -> Dict:
    """문서와 포맷팅된 컨텍스트를 함께 반환
    
    Args:
        question: 검색할 질문

    Returns:
        Dict: 문서와 포맷팅된 컨텍스트, 검색된 문서 리스트
    """

    # 검색 결과 가져오기
    docs = retriever.invoke(question)
    return {
        "question": None,  # 질문
        "context": None,   # 문서 포맷팅된 컨텍스트
        "source_documents": None   # 검색된 문서 리스트
    }

<details>
<summary>💡 정답 보기</summary>

```python
from typing import Dict

def get_context_and_docs(question: str) -> Dict:
    """문서와 포맷팅된 컨텍스트를 함께 반환
    
    Args:
        question: 검색할 질문

    Returns:
        Dict: 문서와 포맷팅된 컨텍스트, 검색된 문서 리스트
    """

    # 검색 결과 가져오기
    docs = retriever.invoke(question)
    return {
        "question": question,
        "context": format_docs(docs),
        "source_documents": docs
    }
```
</details>

`(2) 컨텍스트와 질문을 입력으로 받아 답변을 생성하는 함수`

In [ ]:
# 다음 코드를 완성하세요.

from langchain_core.output_parsers import StrOutputParser

def prompt_and_generate_answer(input_data: Dict) -> Dict:
    """컨텍스트와 질문을 입력으로 받아 답변을 생성

    Args:
        input_data (Dict): 컨텍스트와 질문이 포함된 딕셔너리

    Returns:
        Dict: 생성된 답변과 소스 문서 정보가 포함된 딕셔너리
    """

    # LCEL 체인 구성 (StrOutputParser 사용)
    answer_chain = None

    return {
        "answer": None,  # 생성된 답변 (answer_chain 결과)
        "source_documents": None  # 소스 문서 정보 (input_data에서 가져옴)
    }

<details>
<summary>💡 정답 보기</summary>

```python
from langchain_core.output_parsers import StrOutputParser

def prompt_and_generate_answer(input_data: Dict) -> Dict:
    """컨텍스트와 질문을 입력으로 받아 답변을 생성

    Args:
        input_data (Dict): 컨텍스트와 질문이 포함된 딕셔너리

    Returns:
        Dict: 생성된 답변과 소스 문서 정보가 포함된 딕셔너리
    """

    # LCEL 체인 구성
    answer_chain = prompt | llm | StrOutputParser() #위에서 답변을 엔서에서 리턴을해준다. 리트리버가한거랑 직접 체인에서 리턴하는거랑 다르다.

    return {
        "answer": answer_chain.invoke({
            "context": input_data['context'], 
            "question": input_data['question']
        }),
        "source_documents": input_data['source_documents']
    }
```
</details>

`(3) RAG 체인 구성`

In [ ]:
# 다음 코드를 완성하세요.

from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# Chain 구성
rag_chain = (
    None(get_context_and_docs) |  # 문서와 컨텍스트 가져오기 
    {
        'response': None(prompt_and_generate_answer), # 답변 생성
        'question': None(),
        "source_documents": None("source_documents")   # 소스 반환
    }
)

<details>
<summary>💡 정답 보기</summary>

```python
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# Chain 구성
rag_chain = (
    RunnableLambda(get_context_and_docs) |  # 문서와 컨텍스트 가져오기 
    {
        'response': RunnableLambda(prompt_and_generate_answer), # 답변 생성
        'question': RunnablePassthrough(), #이게아니고 틀린거고 itemgetter("question") 이 정답이다 
        "source_documents": itemgetter("source_documents")   # 소스 반환
    }
)
```
</details>

In [ ]:
# Chain 실행
query = "수원시의 주택건설지역은 어디에 해당하나요?"
result = rag_chain.invoke(query)

# 결과 출력
print("답변:", result["response"]["answer"])
print("\n참조 문서:")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")

### 3) 검색 문서 관련성 평가

`(1) 검색 문서와 질문 간의 관련성을 평가`


In [ ]:
# 검색 문서의 질문 관련성 평가

prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 논리적으로 평가하세요.
단계적으로 진행하며, 평가결과에 대한 검증을 수행하세요.

다음 기준 중 하나 이상을 충족할 경우 'Yes'로 답변하고, 모두 충족하지 못하면 'No'로 답변하세요:

1. 컨텍스트가 질문에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
2. 컨텍스트의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?
3. 컨텍스트의 정보가 질문에 대한 답변을 제공할 수 있는가?

'Yes' 또는 'No'로만 답변하세요."""),
    ("human", """[컨텍스트]
{context}

[질문]
{question}""")
])

chain = prompt | llm | StrOutputParser()    # gpt-4.1-mini 모델 사용

for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")
    relevance = chain.invoke({
        "context": doc.page_content,
        "question": query
    }).lower()

    print(f"평가 결과: {relevance}")

In [ ]:
# gpt-4.1 모델 사용

llm_gpt4 = ChatOpenAI(
    model='gpt-4.1',
    temperature=0.1,
    top_p=0.9, 
)

chain = prompt | llm_gpt4 | StrOutputParser()    # gpt-4.1 모델 사용

for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")
    relevance = chain.invoke({
        "context": doc.page_content,
        "question": query
    }).lower()

    print(f"평가 결과: {relevance}")

##### ***[실습] 문서 관련성 평가 체인을 구조화 출력으로 구현합니다.*** 

- pydantic schema 사용
- with_structured_output 함수 사용

In [ ]:
# 여기에 코드를 작성하세요.

<details>
<summary>💡 정답 보기</summary>

```python
from pydantic import BaseModel, Field
from typing import Literal

class DocumentRelevance(BaseModel):
    """문서 관련성 평가 결과"""
    is_relevant: Literal["yes", "no"] = Field(
        description="문서가 질문에 답변하는데 필요한 정보를 포함하는지 여부"
    )
    reasoning: str = Field(
        description="문서가 질문에 답변하는데 필요한 정보를 포함하는지 여부에 대한 이유"
    )

prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 평가하세요.

다음 기준 중 하나 이상을 충족할 경우 'yes'로, 모두 충족하지 못하면 'no'로 답변하세요:

1. 컨텍스트가 질문에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
2. 컨텍스트의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?"""),
    ("human", """[컨텍스트]
{context}

[질문]
{question}""")
])

structured_llm = llm.with_structured_output(DocumentRelevance)
chain = prompt | structured_llm

for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content}")
    
    evaluation = chain.invoke({
        "context": doc.page_content,
        "question": query
    })
    
    print(f"평가 결과: {evaluation.is_relevant}")
    print(f"이유: {evaluation.reasoning}")
```
</details>

# Gradio 챗봇 인터페이스

- uv add gradio 

In [20]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 벡터 저장소 로드
vector_sotre = Chroma(
    collection_name="housing_faq_db",
    persist_directory="./chroma_db", 
    embedding_function=embeddings,
)


# 검색기 생성 - 유사도 기반 상위 5개 문서 검색
retriever = vector_store.as_retriever(
    search_kwargs={"k": 5},
)

# 테스트 질문
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print("=" * 50)

NameError: name 'vector_store' is not defined

In [ ]:
import gradio as gr
from langchain_core.language_models import BaseChatModel
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from typing import List, Optional
from dataclasses import dataclass

@dataclass
class SearchResult:
    context: str
    source_documents: Optional[List]

class RAGSystem:
    def __init__(
            self, 
            llm: BaseChatModel, 
            eval_llm: BaseChatModel,
            retriever: VectorStoreRetriever
        ):
        if not llm:
            self.llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
        else:
            self.llm = llm

        if not eval_llm:
            self.eval_llm = ChatOpenAI(model="gpt-4.1", temperature=0)
        else:
            self.eval_llm = eval_llm

        if not retriever:
            raise ValueError("검색기(retriever)가 필요합니다.")
        else:
            self.retriever = retriever
        
    def _format_docs(self, docs: List) -> str:
        return "\n\n".join(doc.page_content for doc in docs)
    
    def _format_source_documents(self, docs: Optional[List]) -> str:
        if not docs:
            return "\n\nℹ️ 관련 문서를 찾을 수 없습니다."
        
        formatted_docs = []
        for i, doc in enumerate(docs, 1):
            metadata = doc.metadata if hasattr(doc, 'metadata') else {}
            source_info = []
            
            if 'question_id' in metadata:
                source_info.append(f"ID: {metadata['question_id']}")
            if 'keyword' in metadata:
                source_info.append(f"키워드: {metadata['keyword']}")
            if 'summary' in metadata:
                source_info.append(f"요약: {metadata['summary']}")
                
            formatted_docs.append(
                f"📚 참조 문서 {i}\n"
                f"• {' | '.join(source_info) if source_info else '출처 정보 없음'}\n"
                f"• 내용: {doc.page_content}"
            )
        
        return "\n\n" + "\n\n".join(formatted_docs)
    
    def _check_relevance(self, docs: List, question: str) -> List:
        """문서의 관련성 확인"""

        relevant_docs = []

        if not docs:
            return relevant_docs
            
        prompt = ChatPromptTemplate.from_messages([
            ("system", """주어진 컨텍스트가 질문에 답변하는데 필요한 정보를 포함하고 있는지 평가하세요.

        다음 기준 중 하나 이상을 충족할 경우 'Yes'로 답변하고, 모두 충족하지 못하면 'No'로 답변하세요:

        1. 컨텍스트가 질문에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
        2. 컨텍스트의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?

        'Yes' 또는 'No'로만 답변하세요."""),
            ("human", """[컨텍스트]
        {context}

        [질문]
        {question}""")
        ])
        
        chain = prompt | self.eval_llm | StrOutputParser()

        for doc in docs:
            result = chain.invoke({
                "context": doc.page_content,
                "question": question
            }).lower()

            print(f"문서 {doc.metadata['question_id']} 관련성 확인 결과: {result}")
            print(f"문서 {doc.metadata['question_id']} 내용:")
            print(doc.page_content)
            print("-" * 50)

            if "yes" in result:
                relevant_docs.append(doc)
            
        return relevant_docs
    
    def search_documents(self, question: str) -> SearchResult:
        try:
            docs = retriever.invoke(question)
            print(f"검색된 문서 개수: {len(docs)}")
            relevant_docs = self._check_relevance(docs, question) 
            print(f"관련 문서 개수: {len(relevant_docs)}")
            
            return SearchResult(
                context=self._format_docs(relevant_docs) if relevant_docs else "관련 문서를 찾을 수 없습니다.",
                source_documents=relevant_docs,
            )
        except Exception as e:
            print(f"문서 검색 중 오류 발생: {e}")
            return SearchResult(
                context="문서 검색 중 오류가 발생했습니다.",
                source_documents=None,
            )
    
    def generate_answer(self, message: str, history: List) -> str:
        # 문서 검색
        search_result = self.search_documents(message)
        
        if not search_result.source_documents:
            return "죄송합니다. 관련 문서를 찾을 수 없어 답변하기 어렵습니다. 다른 질문을 해주시겠습니까?"
                    
        # 프롬프트 템플릿 설정
        prompt = ChatPromptTemplate.from_messages([
            ("system", """다음 지침을 따라 질문에 답변해주세요:
            1. 주어진 문서의 내용만을 기반으로 답변하세요.
            2. 문서에 명확한 근거가 없는 내용은 "근거 없음"이라고 답변하세요.
            3. 답변하기 어려운 질문은 "잘 모르겠습니다"라고 답변하세요.
            4. 추측이나 일반적인 지식을 사용하지 마세요."""),
            ("human", "문서들:\n{context}\n\n질문: {question}")
        ])
        
        # RAG Chain 구성
        chain = prompt | self.llm | StrOutputParser()
        
        try:
            # 답변 생성
            answer = chain.invoke({
                "context": search_result.context,
                "question": message
            })
            
            # 참조 문서 포맷팅 추가
            sources = self._format_source_documents(search_result.source_documents)
            return f"{answer}\n{sources}"
            
        except Exception as e:
            return f"답변 생성 중 오류가 발생했습니다: {str(e)}"

# Gradio 인터페이스 설정

rag_system = RAGSystem(
    llm=ChatOpenAI(model="gpt-4.1-mini", temperature=0),   
    eval_llm=ChatOpenAI(model="gpt-4.1", temperature=0),
    retriever=vector_store.as_retriever(search_kwargs={"k": 2})
)

demo = gr.ChatInterface(
    fn=rag_system.generate_answer,
    title="RAG QA 시스템",
    description="""
    질문을 입력하면 관련 문서를 검색하여 답변을 생성합니다.
    모든 답변에는 참조한 문서의 출처가 표시됩니다.
    """,
    examples=[
        ["수원시의 주택건설지역은 어디에 해당하나요?"],
        ["무주택 세대에 대해서 설명해주세요."],
        ["2순위로 당첨된 사람이 청약통장을 다시 사용할 수 있나요?"],
    ],
    analytics_enabled=False,
)

# 데모 실행
demo.launch()

In [ ]:
# Gradio 인터페이스 종료
demo.close()

---

# **[프로젝트 실습] 주택청약 FAQ 시스템 구현**

이전 코드를 기반으로 주택청약 FAQ 시스템을 다음 요구사항에 맞춰 개선합니다. 

- 생성된 답변의 품질을 평가 (답변이 불충분한 경우 예외 처리)
- 관련성 높은 FAQ 문서 검색 (임베딩 모델, 청크 크기, 벡터 검색 방법 등)

제약 조건:
- Gradio ChatInterface 사용
- RAG 구조 유지

In [ ]:
# 여기에 코드를 작성하세요. 